In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import random
from summer2.utils import Epoch
import networkx as nx
import numpy as np

In [ ]:
def get_random_dates(start_time, n_cases, epidemic_duration):
    """Generate random onset dates for cases.
    """
    date_ints = random.sample(range(epidemic_duration), n_cases)
    return [start_time + timedelta(days=i_time) for i_time in date_ints]

def get_dates_df_from_list(dates_list):
    """Dataframe with every subsequent case infected by the first one.
    """
    case_data_df = pd.DataFrame(sorted(dates_list), columns=["date"])
    case_data_df["infector"] = 0
    return case_data_df

def generate_random_infectors(cases_df):
    """Randomly assign each case a preceding infector.
    """
    for i_case in cases_df.index[1:]:
        cases_df.loc[i_case, "infector"] = random.randint(0, i_case - 1)

def get_infector_graph(df):
    """Plot infection process graph from dataframe with standard format.
    """
    graph = nx.DiGraph()
    graph.add_nodes_from(df.index)
    graph.add_edges_from(list(zip(df["infector"], df.index))[1:])
    return graph

def draw_infector_graph(df, graph, epoch):
    """Draw the graph created by the previous function.
    """
    positions = [(n, 0) for n in epoch.dti_to_index(df["date"])]
    nx.draw(graph, positions, with_labels=True, connectionstyle="arc3, rad=1", node_size=400)

In [ ]:
epoch = Epoch(datetime(2019, 12, 31))
case_dates = get_random_dates(datetime(2020, 1, 1), 10, 400)
case_data_df = get_dates_df_from_list(case_dates)
generate_random_infectors(case_data_df)
graph = get_infector_graph(case_data_df)
draw_infector_graph(case_data_df, graph, epoch)

In [ ]:
rng = np.random.default_rng(0)
graph = nx.DiGraph()

In [ ]:
epoch = Epoch(datetime(2019, 12, 31))
duration = 10
case_series = np.concatenate([np.array([1]), rng.integers(low=2, high=5, size=duration)])

In [ ]:
positions = []

In [ ]:
graph = nx.DiGraph()
for date_int, new_cases in enumerate(case_series):
    prev_graph_size = len(graph)
    new_graph_size = len(graph) + new_cases
    new_case_ids = range(prev_graph_size, new_graph_size)
    positions += [(date_int, 0)] * len(new_case_ids)
    graph.add_nodes_from(new_case_ids, time=date_int, date=epoch.number_to_datetime(date_int))
    if date_int > 0:
        infectors = rng.choice(np.arange(prev_graph_size), new_cases)
        graph.add_edges_from(zip(infectors, new_case_ids))

nx.draw(graph, positions, with_labels=True, connectionstyle="arc3, rad=1", node_size=400)

In [ ]:
case = 10
nx.get_node_attributes(graph.subgraph(nx.ancestors(graph, case) | {case}),"date")